In [1]:
from mysql_collect_test import connect_config
from driver.collector.collector_factory import get_collector
from driver.collector.mysql_collector import MysqlCollector
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

QUERY_EXAMPLE = """select * from sys.statement_analysis where query is not NULL and query <> 'COMMIT' order by exec_count desc limit 1;"""

config = connect_config()

def q(query):
    with get_collector(config) as collector:
       # print(collector._cmd(query))  # 0 data 1 meta
       
        df = pd.read_sql("select * from sys.statement_analysis where query is not NULL and query <> 'COMMIT' order by exec_count desc limit 1;", con=collector.get_conn())
        #print(df)

        from IPython.core.display import HTML
        display(HTML(df.to_html()))

    
q(QUERY_EXAMPLE)


,query,db,full_scan,exec_count,err_count,warn_count,total_latency,max_latency,avg_latency,lock_latency,cpu_latency,rows_sent,rows_sent_avg,rows_examined,rows_examined_avg,rows_affected,rows_affected_avg,tmp_tables,tmp_disk_tables,rows_sorted,sort_merge_passes,digest,first_seen,last_seen
0,SET NAMES ? COLLATE ?,information_schema,,33,0,66,5.57 ms,987.76 us,168.77 us,0 ps,0 ps,0,0.0,0,0.0,0,0.0,0,0,0,0,52eaff6a7819ec096aa8c71501acd23ef138f2a775fc2827cc34ed7d77fa3e3d,2022-09-01 12:54:21.115362,2022-09-01 20:08:46.885131


In [2]:
%matplotlib inline
%matplotlib widget
#%matplotlib notebook

from matplotlib import pyplot as plt
from matplotlib.container import BarContainer

import numpy as np

import pandas as pd


import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact, interact_manual, Layout


In [4]:
from query_library import get_query_library

ql = get_query_library()

w1 = widgets.Dropdown(
    options=list(ql.keys()),
    #description='Metric type:',
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(w1.value)
        w2 = widgets.Dropdown(
        options=list(ql[w1.value].keys()),
        #description='Metric type:',
        )
        box = HBox([w1,w2])
        display(box)
        print("changed to %s" % change['new'])
        
w1.observe(on_change)

box = HBox([w1,]) 
display(box)

In [12]:
x_widget = widgets.Dropdown(options = list(ql.keys()))
y_widget = widgets.Dropdown()

# Define a function that updates the content of y based on what we select for x
def update(*args):
    y_widget.options = list(ql[x_widget.value].keys())
x_widget.observe(update)

# Some function you want executed
# def random_function():
#     print(x_widget.value)

# interact(random_function,
#          x = x_widget,
#          y = y_widget);
def on_change(change):
    print(ql[x_widget.value][y_widget.value])
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

y_widget.observe(on_change)


display(x_widget)
display(y_widget)



Dropdown(options=('data', 'index', 'Host', 'User', 'Object', 'Performance', 'Cluster', 'Statements', 'Table_St…

Dropdown(options=(), value=None)

In [11]:
print(x_widget.value)
print(y_widget.value)

print(ql[x_widget.value][y_widget.value])

Host
Host_Hitting_by_Tablescans
SELECT host
    FROM sys.x$host_summary
    ORDER BY table_scans DESC LIMIT 5;


In [15]:
import pandas
import pymysql
from datetime import datetime

print('START TIME : ',str(datetime.now())[10:19] )

conn=pymysql.connect(host='localhost',port=3306,user='root',password='',db='information_schema')
sql="select * from index_test"

result = pandas.read_sql_query(QUERY_EXAMPLE,conn)
print(result)

print('END TIME : ',str(datetime.now())[10:19] )

conn.close()

START TIME :   13:29:32


/usr/local/lib/python3.10/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'select * from sys.statement_analysis where query is not NULL and query <> 'COMMIT' order by exec_count desc limit 1;': (1267, "Illegal mix of collations (utf8mb4_0900_ai_ci,COERCIBLE) and (utf8mb4_general_ci,COERCIBLE) for operation '<>'")